In [5]:
import pandas as pd
import pyodbc
import sqlite3
import numpy as np
from datetime import datetime, timedelta

DB = {'servername': r'LAPTOP-NBUM6TMN',
    'database': 'johari'}

ssms_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                     ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
ssms_cursor = ssms_conn.cursor()

# Maak een connectie met de go_staff- en go_sales-databases
go_staff_conn = sqlite3.connect(r"../../week_2/sqlite/go_staff.sqlite")
go_sales_conn = sqlite3.connect(r"../../week_2/sqlite/go_sales.sqlite")


# Lijst met tabelnamen van de go_staff-database
go_staff_table_names = ['course', 'sales_branch', 'sales_staff', 'satisfaction', 'satisfaction_type', 'training']
go_sales_table_names = ['country', 'order_details', 'order_header', 'order_method', 'product', 'product_line', 'product_type', 'retailer_site', 'return_reason', 'returned_item', 'sales_branch', 'sales_staff', 'SALES_TARGETData']

# Dictionary om de dataframes op te slaan
go_staff = {}
go_sales = {}

# Lees de tabellen in de go_staff-database in
for table_name in go_staff_table_names:
    go_staff[table_name] = pd.read_sql_query(f"SELECT * FROM {table_name}", go_staff_conn)

# Lees de tabellen in de go_sales-database in
for table_name in go_sales_table_names:
    go_sales[table_name] = pd.read_sql_query(f"SELECT * FROM {table_name}", go_sales_conn)

def read_source_table(conn, table_name):
    query = f'SELECT * FROM {table_name}'
    df = conn.execute(query)
    result = pd.DataFrame(df.fetchall(), columns=[column[0] for column in df.description])
    return result

def read_source_tables(conn_sales, conn_country, table_names):
    dfs = {}
    for table_name in table_names:
        query = f'SELECT * FROM {table_name}'
        df = pd.read_sql_query(query, conn_sales)
        dfs[table_name] = df

    # Read country table from different database
    query = 'SELECT * FROM country'
    df_country = pd.read_sql_query(query, conn_country)
    dfs['country'] = df_country

    # Merge sales_staff with sales_branch
    result = pd.merge(dfs['sales_staff'], dfs['sales_branch'], how='inner', left_on='SALES_BRANCH_CODE', right_on='SALES_BRANCH_CODE')

    # Merge the result with country
    result = pd.merge(result, dfs['country'], how='inner', left_on='COUNTRY_CODE', right_on='COUNTRY_CODE')

    return result

def clean_target_table(conn, table_name):
    ssms_cursor.execute(f'DELETE FROM {table_name}')
    ssms_conn.commit()

def course_etl():
    # Extract
    course = read_source_table(go_staff_conn, 'course')

    # Clean Target
    clean_target_table(ssms_conn, 'COURSE')
    # Sla de gegevens binnen de course-tabel op in de SQL Server-database
    for index, row in course.iterrows():
        try:
            #Transform
            course_id = row['COURSE_CODE']
            course_description = row['COURSE_DESCRIPTION']   

            #Load
            query = f"INSERT INTO dbo.COURSE (course_id, course_description) VALUES ({course_id}, '{course_description}')"
            ssms_cursor.execute(query)
            ssms_conn.commit()

        except Exception as e:
            print(e)
            break

def satisfaction_type_etl():
    satisfaction_type = read_source_table(go_staff_conn, 'satisfaction_type')

    # Clean Target
    clean_target_table(ssms_conn, 'rsatisfaction_type')

    for index, row in satisfaction_type.iterrows():
        try:
            satisfaction_type_id = row['SATISFACTION_TYPE_CODE']
            satisfaction_description = row['SATISFACTION_TYPE_DESCRIPTION']   

            query = f"INSERT INTO dbo.rsatisfaction_type (satisfaction_type_id, satisfaction_description) VALUES ({satisfaction_type_id}, '{satisfaction_description}')"

            ssms_cursor.execute(query)
            ssms_conn.commit()

        except Exception as e:
            print(e)
            break

def sales_staff_etl():
    table_names = ['sales_staff', 'sales_branch']
    sales_staff = read_source_tables(go_staff_conn, go_sales_conn, table_names)

    # Clean Target
    clean_target_table(ssms_conn, 'rSALES_STAFF')

    for index, row in sales_staff.iterrows():
        try:
            sales_staff_id = row['SALES_STAFF_CODE']
            sales_staff_first_name = row['FIRST_NAME']
            sales_staff_last_name = row['LAST_NAME']
            sales_staff_phone = row['WORK_PHONE']
            sales_staff_email = row['EMAIL']
            sales_staff_fax = row['FAX']
            sales_staff_sales_branch_id = row['SALES_BRANCH_CODE']
            sales_staff_position_en = row['POSITION_EN']
            sales_staff_date_hired = row['DATE_HIRED']
            sales_staff_extension = row['EXTENSION']
            sales_staff_sales_branch_address1 = row['ADDRESS1']
            sales_staff_sales_branch_address2 = row['ADDRESS2']
            sales_staff_sales_branch_city = row['CITY']
            sales_staff_sales_branch_region = row['REGION']
            sales_staff_sales_branch_postal_zone = row['POSTAL_ZONE']
            sales_staff_country_id = row['COUNTRY_CODE']
            sales_staff_country_name = row['COUNTRY']
            sales_staff_country_language = row['LANGUAGE']
            sales_staff_country_currency = row['CURRENCY_NAME']

            query = """
                INSERT INTO rSALES_STAFF (
                    sales_staff_id, 
                    sales_staff_first_name, 
                    sales_staff_last_name, 
                    sales_staff_phone, 
                    sales_staff_email, 
                    sales_staff_fax, 
                    sales_staff_sales_branch_id, 
                    sales_staff_position_en, 
                    sales_staff_date_hired, 
                    sales_staff_extension, 
                    sales_staff_sales_branch_address1, 
                    sales_staff_sales_branch_address2, 
                    sales_staff_sales_branch_city, 
                    sales_staff_sales_branch_region, 
                    sales_staff_sales_branch_postal_zone, 
                    sales_staff_country_id, 
                    sales_staff_country_name, 
                    sales_staff_country_language, 
                    sales_staff_country_currency
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """

            params = (
                sales_staff_id, 
                sales_staff_first_name, 
                sales_staff_last_name, 
                sales_staff_phone, 
                sales_staff_email, 
                sales_staff_fax, 
                sales_staff_sales_branch_id, 
                sales_staff_position_en, 
                sales_staff_date_hired, 
                sales_staff_extension, 
                sales_staff_sales_branch_address1, 
                sales_staff_sales_branch_address2, 
                sales_staff_sales_branch_city, 
                sales_staff_sales_branch_region, 
                sales_staff_sales_branch_postal_zone, 
                sales_staff_country_id, 
                sales_staff_country_name, 
                sales_staff_country_language, 
                sales_staff_country_currency
            )

            ssms_cursor.execute(query, params)
            ssms_conn.commit()

        except Exception as e:
            print(e)
            break

def country():
    country = read_source_table(go_sales_conn, 'country')

    # Clean Target
    clean_target_table(ssms_conn, 'COUNTRY')

    for index, row in country.iterrows():
        try:
            country_id = row['COUNTRY_CODE']
            country_name = row['COUNTRY']
            country_language = row['LANGUAGE']
            country_currency = row['CURRENCY_NAME']

            query = f"INSERT INTO dbo.COUNTRY VALUES ('{country_id}', '{country_name}', '{country_language}', '{country_currency}')"

            ssms_cursor.execute(query)
            ssms_conn.commit()

        except Exception as e:
            print(e)
            break

def order_method():
    order_method = read_source_table(go_sales_conn, 'order_method')

    # Clean Target
    clean_target_table(ssms_conn, 'ORDER_METHOD')

    for index, row in order_method.iterrows():
        try:
            order_method_id= row['ORDER_METHOD_CODE']
            order_method_en = row['ORDER_METHOD_EN']

            query = f"INSERT INTO dbo.ORDER_METHOD VALUES ('{order_method_id}', '{order_method_en}')"

            ssms_cursor.execute(query)
            ssms_conn.commit()

        except Exception as e:
            print(e)
            break

def get_product_surrogate_key(product_number):
    query = f"SELECT PRODUCT_SK FROM rPRODUCT WHERE product_product_id = '{product_number}'"
    ssms_cursor.execute(query)
    result = ssms_cursor.fetchone()
    return result[np.random.randint(0, len(result))]

def get_product_surrogate_key_and_check_date(product_number, date):
    query = f"SELECT PRODUCT_SK FROM rPRODUCT WHERE product_product_id = '{product_number}' AND product_timestamp <= '{date}'"
    ssms_cursor.execute(query)
    result = ssms_cursor.fetchone()

    if result is None:
        return get_product_surrogate_key(product_number)
    else:
        return result[np.random.randint(0, len(result))]

def orders():
    merged_order = pd.merge(go_sales['order_details'], go_sales['order_header'], on='ORDER_NUMBER')
    merged_order = pd.merge(merged_order, go_sales['product'], on='PRODUCT_NUMBER')

    testdb = sqlite3.connect(r"test.sqlite")
    merged_order.to_sql('ORDER', testdb, if_exists='replace', index=False)

    # Clean Target
    clean_target_table(ssms_conn, 'ORDERS')

    for index, row in merged_order.iterrows():
        try:
            order_id = row['ORDER_DETAIL_CODE']
            product_id = row['PRODUCT_NUMBER']
            order_product_sk = get_product_surrogate_key(product_id)
            staff_id = row['SALES_STAFF_CODE']
            method_id = row['ORDER_METHOD_CODE']
            retailer_id = row['RETAILER_SITE_CODE']
            quantity = row['QUANTITY']
            unit_cost = float(pd.to_numeric(row['UNIT_COST']))
            unit_sale = float(pd.to_numeric(row['UNIT_SALE_PRICE']))
            product_margin = row['MARGIN']
            product_cost = float(pd.to_numeric(row['UNIT_PRICE']))

            # Afgeleide dimensies
            total_price_before_sales = product_cost * quantity
            total_sale_amount = (product_cost - unit_sale) * quantity
            total_price_after_sales = total_price_before_sales - total_sale_amount
            total_profit = (unit_sale - unit_cost) * quantity

            timestamp = pd.to_datetime(row['ORDER_DATE'])

            query = "INSERT INTO dbo.ORDERS VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
            params = (order_id, product_id, order_product_sk, staff_id, method_id, retailer_id, quantity, unit_cost, unit_sale, product_margin, product_cost, total_price_before_sales, total_sale_amount, total_price_after_sales, total_profit, timestamp)

            ssms_cursor.execute(query, params)
            ssms_conn.commit()
        except Exception as e:
            print(e)
            print(retailer_id)
            break

def product():
    product = read_source_table(go_sales_conn, 'product')
    merged_product = pd.merge(product, go_sales['product_type'], on='PRODUCT_TYPE_CODE')
    merged_product = pd.merge(merged_product, go_sales['product_line'], on='PRODUCT_LINE_CODE')

    # Clean Target
    clean_target_table(ssms_conn, 'rPRODUCT')

    for index, row in merged_product.iterrows():
        try:
            product_id = row['PRODUCT_NUMBER']
            product_name = row['PRODUCT_NAME']
            product_image = row['PRODUCT_IMAGE']
            product_line_name = row['PRODUCT_LINE_EN']
            product_type_name = row['PRODUCT_TYPE_CODE']
            product_introduction_date = pd.to_datetime(row['INTRODUCTION_DATE'], format='%d-%m-%Y')            
            product_cost = row['PRODUCTION_COST']
            product_margin = row['MARGIN']
            product_language = row['LANGUAGE']
            
            timestamp = datetime.now() - timedelta(days=np.random.randint(0, 365*5), hours=np.random.randint(0, 24))
            while timestamp < product_introduction_date:
                timestamp = datetime.now() - timedelta(days=np.random.randint(0, 365*5), hours=np.random.randint(0, 24))
            
            query = "INSERT INTO dbo.rPRODUCT VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
            params = (product_id, product_name, product_image, product_line_name, product_type_name, product_introduction_date, product_cost, product_margin, product_language, timestamp)

            ssms_cursor.execute(query, params)
            ssms_conn.commit()
        except Exception as e:
            print(e)
            break

def simulate_surrogate_key():
    # Selecteer een willekeurige rij uit de rPRODUCT-tabel en INSERT een nieuwe rij met dezelfde gegevens, maar met een nieuwe SURROGATE KEY
    query = "SELECT TOP 1 * FROM dbo.rPRODUCT ORDER BY NEWID()"
    ssms_cursor.execute(query)
    row = ssms_cursor.fetchone()
    
    query = "INSERT INTO dbo.rPRODUCT VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    params = (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], datetime.now() + timedelta(days=1, hours=3))

    ssms_cursor.execute(query, params)
    ssms_conn.commit()

    # Selecteer de nieuw toegevoegde rij en INSERT een nieuwe rij binnen de ORDERS-tabel met een FOREIGN SURROGATE KEY naar de SURROGATE KEY van de nieuwe rPRODUCT-rij
    query = "SELECT TOP 1 * FROM dbo.rPRODUCT ORDER BY PRODUCT_SK DESC"
    ssms_cursor.execute(query)
    row = ssms_cursor.fetchone()
    pk = row[0]
    sk = row[10]

    query = "SELECT TOP 1 * FROM dbo.ORDERS WHERE ORDER_PRODUCT_id = ?"
    params = (pk)
    ssms_cursor.execute(query, params)
    row = ssms_cursor.fetchone()

    random_order_id = np.random.randint(1, 1000)

    # Nieuwe timestamp voor de order
    timestamp = row[15] + timedelta(days=1, hours=3)

    try: 
        query = "INSERT INTO dbo.ORDERS VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
        params = (random_order_id, pk, sk, row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12], row[13], row[14], timestamp)
        ssms_cursor.execute(query, params)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        print(row)

def sales_targetdata():
    sales_targetdata = read_source_table(go_sales_conn, 'SALES_TARGETData')

    # Clean Target
    clean_target_table(ssms_conn, 'SALES_TARGETData')

    for index, row in sales_targetdata.iterrows():
        try:
            target_id = row['Id']
            product_id = row['PRODUCT_NUMBER']
            product_fsk = get_product_surrogate_key_and_check_date(product_id, datetime(int(row['SALES_YEAR']), int(row['SALES_PERIOD']), 1))     
            staff_id = row['SALES_STAFF_CODE']
            retailer_code = row['RETAILER_CODE']
            sales_year_and_period = datetime(int(row['SALES_YEAR']), int(row['SALES_PERIOD']), 1)
            sales_target = row['SALES_TARGET']
            
            query = "INSERT INTO dbo.SALES_TARGETData VALUES (?, ?, ?, ?, ?, ?, ?)"
            params = (target_id, product_id, product_fsk, staff_id, retailer_code, sales_year_and_period, sales_target)

            ssms_cursor.execute(query, params)
            ssms_conn.commit()
        except Exception as e:
            print(e)
            print(row)
            break

def return_reason():
    return_reason = read_source_table(go_sales_conn, 'return_reason')

    # Clean Target
    clean_target_table(ssms_conn, 'RETURN_REASON')

    for index, row in return_reason.iterrows():
        try:
            reason_id = row['RETURN_REASON_CODE']
            reason_description = row['RETURN_DESCRIPTION_EN']

            query = "INSERT INTO dbo.RETURN_REASON VALUES (?, ?)"
            params = (reason_id, reason_description)

            ssms_cursor.execute(query, params)
            ssms_conn.commit()
        except Exception as e:
            print(e)
            print(row)
            break

def returned_item():
    returned_item = read_source_table(go_sales_conn, 'returned_item')
    returned_item = pd.merge(returned_item, go_sales['return_reason'], on=['RETURN_REASON_CODE'])
    returned_item = pd.merge(returned_item, go_sales['order_details'], on=['ORDER_DETAIL_CODE'])

    # Clean Target
    clean_target_table(ssms_conn, 'RETURNED_ITEM')

    for index, row in returned_item.iterrows():
        try:
            return_id = row['RETURN_CODE']
            reason_id = row['RETURN_REASON_CODE']
            reason_description = row['RETURN_DESCRIPTION_EN']
            return_date = pd.to_datetime(row['RETURN_DATE'], dayfirst=True)
            order_detail_id = row['ORDER_DETAIL_CODE']
            return_quantity = row['RETURN_QUANTITY']
            product_id = row['PRODUCT_NUMBER']
            product_fsk = get_product_surrogate_key(product_id)

            query = "INSERT INTO dbo.RETURNED_ITEM VALUES (?, ?, ?, ?, ?, ?, ?, ?)"
            params = (return_id, reason_id, reason_description, return_date, order_detail_id, return_quantity, product_id, product_fsk)

            ssms_cursor.execute(query, params)
            ssms_conn.commit()
        except Exception as e:
            print(e)
            print(row)
            break

# course_etl()
# satisfaction_type_etl()
# sales_staff_etl()
# country()
# order_method()
# product()
# orders()
# simulate_surrogate_key()
sales_targetdata()
# return_reason()
# returned_item()

ssms_conn.close()

In [8]:
ssms_conn.close()

In [24]:
order_details_columns = read_source_table(go_sales_conn, 'order_details').columns
print(order_details_columns)

Index(['ORDER_DETAIL_CODE', 'ORDER_NUMBER', 'PRODUCT_NUMBER', 'QUANTITY',
       'UNIT_COST', 'UNIT_PRICE', 'UNIT_SALE_PRICE', 'TRIAL879'],
      dtype='object')


In [9]:
import pandas as pd
import pyodbc
import sqlite3
import numpy as np

DB = {'servername': r'LAPTOP-NBUM6TMN',
    'database': 'johari'}

ssms_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                     ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
ssms_cursor = ssms_conn.cursor()


go_crm_conn = sqlite3.connect(r"..\..\week_2\sqlite\go_crm.sqlite")
# Lijst met tabelnamen van de go_sales-database
go_crm_table_names = ['age_group', 'country', 'retailer', 'retailer_contact', 'retailer_headquarters', 'retailer_segment', 'retailer_site', 'retailer_type', 'sales_demographic', 'sales_territory']

# Dictionary om de dataframes op te slaan
go_crm = {}

def read_source_table(conn, table_name):
    query = f'SELECT * FROM {table_name}'
    df = conn.execute(query)
    result = pd.DataFrame(df.fetchall(), columns=[column[0] for column in df.description])
    return result

def read_source_tables(conn_crm, table_names):
    dfs = {}
    for table_name in table_names:
        query = f'SELECT * FROM {table_name}'
        df = pd.read_sql_query(query, conn_crm)
        if 'TRIAL222' in df.columns:
            df = df.rename(columns={'TRIAL222': f'TRIAL222_{table_name}'})
        dfs[table_name] = df

    result = pd.merge(dfs['retailer_contact'], dfs['retailer_site'], how='inner', left_on='RETAILER_SITE_CODE', right_on='RETAILER_SITE_CODE')

    result = pd.merge(result, dfs['retailer'], how='inner', left_on='RETAILER_CODE', right_on='RETAILER_CODE')
    result = pd.merge(result, dfs['country'], how='inner', left_on='COUNTRY_CODE', right_on='COUNTRY_CODE')
    result = pd.merge(result, dfs['retailer_headquarters'], how='inner', left_on='RETAILER_CODEMR', right_on='RETAILER_CODEMR')
    result = pd.merge(result, dfs['retailer_segment'], how='inner', left_on='SEGMENT_CODE', right_on='SEGMENT_CODE')
    result = pd.merge(result, dfs['retailer_type'], how='inner', left_on='RETAILER_TYPE_CODE', right_on='RETAILER_TYPE_CODE')
    result = pd.merge(result, dfs['sales_demographic'], how='inner', left_on='RETAILER_CODEMR', right_on='RETAILER_CODEMR')
    result = pd.merge(result, dfs['age_group'], how='inner', left_on='AGE_GROUP_CODE', right_on='AGE_GROUP_CODE')
    result = pd.merge(result, dfs['sales_territory'], how='inner', left_on='SALES_TERRITORY_CODE', right_on='SALES_TERRITORY_CODE')

    testdb = sqlite3.connect(r"test.sqlite")
    result.to_sql('retailer_contact', testdb, if_exists='replace', index=False)
    return result

def clean_target_table(conn, table_name):
    ssms_cursor.execute(f'DELETE FROM {table_name}')
    ssms_conn.commit()

def retailer_contact_etl():
    retailer_contact = read_source_tables(go_crm_conn, ['age_group', 'country', 'retailer', 'retailer_contact', 'retailer_headquarters', 'retailer_segment', 'retailer_site', 'retailer_type', 'sales_demographic', 'sales_territory'])
    
    # Clean Target
    clean_target_table(ssms_conn, 'RETAILER_CONTACT')

    

    for index, row in retailer_contact.iterrows():
        try:
            query = """
                INSERT INTO dbo.RETAILER_CONTACT (
                    retailer_contact_contact_id, 
                    retailer_contact_site_id, 
                    retailer_contact_first_name, 
                    retailer_contact_last_name, 
                    retailer_contact_job_position_en, 
                    retailer_contact_extension, 
                    retailer_contact_fax, 
                    retailer_contact_email, 
                    retailer_contact_gender, 
                    retailer_id, 
                    retailer_codemr, 
                    retailer_company_name, 

                    retailer_age_group_id, 
                    retailer_age_group_upper_age, 
                    retailer_age_group_lower_age, 
                    retailer_sales_demographic_id, 
                    retailer_sales_demographic_codemr, 
                    retailer_sales_demographic_age_group_id, 
                    retailer_sales_demographic_sales_percent, 
                    retailer_sales_territory_id, 
                    retailer_sales_territory_name_en, 
                    retailer_country_id, 
                    retailer_country_country_en, 
                    retailer_country_flag_image, 
                    retailer_country_sales_territory_id, 
                    retailer_site_id, 
                    retailer_site_retailer_id, 
                    retailer_site_address1, 
                    retailer_site_address2, 
                    retailer_site_city, 
                    retailer_site_region, 
                    retailer_site_postal_zone, 
                    retailer_site_country_id, 
                    retailer_site_active_indicator, 
                    retailer_headquarters_codemr, 
                    retailer_headquarters_name, 
                    retailer_headquarters_address1, 
                    retailer_headquarters_address2, 
                    retailer_headquarters_city, 
                    retailer_headquarters_region,  
                    retailer_headquarters_postal_zone, 
                    retailer_headquarters_country_id, 
                    retailer_headquarters_phone, 
                    retailer_headquarters_fax, 
                    retailer_headquarters_segment_id, 
                    retailer_type_id, 
                    retailer_type_type_en, 
                    retailer_segment_id, 
                    retailer_segment_language, 
                    retailer_segment_name, 
                    retailer_segment_description 
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """
            
            params = (
                row['RETAILER_CONTACT_CODE'],
                row['RETAILER_SITE_CODE'],
                row['FIRST_NAME'],
                row['LAST_NAME'],
                row['JOB_POSITION_EN'],
                row['EXTENSION'],
                row['FAX_x'],
                row['E_MAIL'],
                row['GENDER'],
                row['RETAILER_CODE'],
                row['RETAILER_CODEMR'],
                row['COMPANY_NAME'],
                #row['RETAILER_TYPE_CODE'],
                row['AGE_GROUP_CODE'],
                row['UPPER_AGE'],
                row['LOWER_AGE'],
                row['DEMOGRAPHIC_CODE'],
                row['RETAILER_CODEMR'],
                row['AGE_GROUP_CODE'],
                row['SALES_PERCENT'],
                row['SALES_TERRITORY_CODE'],
                row['TERRITORY_NAME_EN'],
                row['COUNTRY_CODE_x'],
                row['COUNTRY_EN'],
                row['FLAG_IMAGE'],
                row['SALES_TERRITORY_CODE'],
                row['RETAILER_SITE_CODE'],
                row['RETAILER_CODE'],
                row['ADDRESS1_x'],
                row['ADDRESS2_x'],
                row['CITY_x'],
                row['REGION_x'],
                row['POSTAL_ZONE_x'],
                row['COUNTRY_CODE_x'],
                row['ACTIVE_INDICATOR'],
                row['RETAILER_CODEMR'],
                row['RETAILER_NAME'],
                row['ADDRESS1_y'],
                row['ADDRESS2_y'],
                row['CITY_y'],
                row['REGION_y'],
                row['POSTAL_ZONE_y'],
                row['COUNTRY_CODE_y'],
                row['PHONE'],
                row['FAX_y'],
                row['SEGMENT_CODE'],
                row['RETAILER_TYPE_CODE'],
                row['RETAILER_TYPE_EN'],
                row['SEGMENT_CODE'],
                row['LANGUAGE'],
                row['SEGMENT_NAME'],
                row['SEGMENT_DESCRIPTION']
            )
            ssms_cursor.execute(query, params)
            ssms_conn.commit()

        except Exception as e:
            print(e)
            break

retailer_contact_etl()

ssms_conn.close()